In [3]:
from itertools import chain
import nltk
from sklearn.metrics import classification_report, f1_score
from sklearn.preprocessing import LabelBinarizer
from collections import Counter
import pickle as pickle
import numpy as np
from pycrfsuite import Trainer, Tagger
from flexcrf_tp.crfsuite2flexcrf import convert_data_to_flexcrf
import warnings
from flexcrf_tp.models.linear_chain import (_feat_fun_values,
                                            _compute_all_potentials,
                                            _forward_score,
                                            _backward_score,
                                            _partition_fun_value,
                                            _posterior_score)

warnings.filterwarnings("ignore")

print(nltk.corpus.conll2002.fileids())
print(nltk.corpus.conll2000.fileids())

['esp.testa', 'esp.testb', 'esp.train', 'ned.testa', 'ned.testb', 'ned.train']
['train.txt', 'test.txt']


In [4]:
def feat(sent, i, pos=''):
    word = sent[i][0]
    postag = sent[i][1]
    return [
        pos + 'word.lower=' + word.lower(),
        pos + 'word.istitle=%s' % word.istitle(),
        pos + 'word.isupper=%s' % word.isupper(),
        pos + 'word[-3:]=' + word[-3:],
        pos + 'word[-2:]=' + word[-2:],
        pos + 'word.isdigit=%s' % word.isdigit(),
        pos + 'postag=' + postag,
        pos + 'postag[:2]=' + postag[:2]]

def word2features(sent, i):
    l = len(sent)
    features = ['bias']
    features.extend(feat(sent, i))
    features.extend(feat(sent, i-1, '-1:') if i > 0 else ['BOS'])
    if i > 1: features.extend(feat(sent, i-2, '-2:'))
    if i > 2: features.extend(feat(sent, i-3, '-3:'))
    if i < l-3: features.extend(feat(sent, i+3, '+3:'))
    if i < l-2: features.extend(feat(sent, i+2, '+2:'))
    features.extend(feat(sent, i+1, '+1') if i < l-1 else ['EOS'])
    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [5]:
train_sents = list(nltk.corpus.conll2002.iob_sents('esp.train'))
test_sents = list(nltk.corpus.conll2002.iob_sents('esp.testb'))

X_train = [sent2features(s) for s in train_sents]
y_train = [sent2labels(s) for s in train_sents]
X_test = [sent2features(s) for s in test_sents]
y_test = [sent2labels(s) for s in test_sents]

trainer = Trainer(verbose=False)
for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

In [6]:
def bio_classification_report(y_true, y_pred):
    """
    Classification report for a list of BIO-encoded sequences.
    It computes token-level metrics and discards "O" labels.

    Note that it requires scikit-learn 0.15+ (or a version from github master)
    to calculate averages properly!
    """
    lb = LabelBinarizer()
    y_true_combined = lb.fit_transform(list(chain.from_iterable(y_true)))
    y_pred_combined = lb.transform(list(chain.from_iterable(y_pred)))

    tagset = set(lb.classes_) - {'O'}
    tagset = sorted(tagset, key=lambda tag: tag.split('-', 1)[::-1])
    class_indices = {cls: idx for idx, cls in enumerate(lb.classes_)}

    return classification_report(
        y_true_combined,
        y_pred_combined,
        labels=[class_indices[cls] for cls in tagset],
        target_names=tagset)

In [7]:
from itertools import product
from multiprocessing import Pool
import numpy as np

def tag(c1_c2):
    c1, c2 = c1_c2
    trainer = Trainer(
        params={
            'c1': c1, 'c2': c2, 'max_iterations': 50
        }, verbose=False)
    
    for xseq, yseq in zip(X_train, y_train):
        trainer.append(xseq, yseq)

    filename = 'conll2002-esp_{}_{}.crfsuite'.format(c1, c2) 
    trainer.train(filename)

    tagger = Tagger()
    tagger.open(filename)

    y_pred = [tagger.tag(xseq) for xseq in X_test]
    return c1, c2, y_pred

In [8]:
c1s = np.logspace(-2, 0, 4)
c2s = np.logspace(-2, 0, 4)
c1_c2s = product(c1s, c2s)

with Pool() as p:
    res = p.map(tag, c1_c2s)

In [12]:
for (c1, c2, y_pred) in res:
    lb = LabelBinarizer()
    y_true_combined = lb.fit_transform(list(chain.from_iterable(y_test)))
    y_pred_combined = lb.transform(list(chain.from_iterable(y_pred)))

    tagset = set(lb.classes_) - {'O'}
    tagset = sorted(tagset, key=lambda tag: tag.split('-', 1)[::-1])
    class_indices = {cls: idx for idx, cls in enumerate(lb.classes_)}

    f1score = f1_score(y_true_combined,y_pred_combined,
        labels=[class_indices[cls] for cls in tagset], average='micro')
    print('c1:{0:.3f}, c2:{1:.3f} :\tf1_score:{2:.4f}'.format(c1, c2, f1score))
#     print(bio_classification_report(y_test, y_pred))

c1:0.010, c2:0.010 :	f1_score:0.8057
c1:0.010, c2:0.046 :	f1_score:0.8023
c1:0.010, c2:0.215 :	f1_score:0.8070
c1:0.010, c2:1.000 :	f1_score:0.8047
c1:0.046, c2:0.010 :	f1_score:0.7995
c1:0.046, c2:0.046 :	f1_score:0.7968
c1:0.046, c2:0.215 :	f1_score:0.8072
c1:0.046, c2:1.000 :	f1_score:0.8040
c1:0.215, c2:0.010 :	f1_score:0.8051
c1:0.215, c2:0.046 :	f1_score:0.8060
c1:0.215, c2:0.215 :	f1_score:0.8064
c1:0.215, c2:1.000 :	f1_score:0.7986
c1:1.000, c2:0.010 :	f1_score:0.7915
c1:1.000, c2:0.046 :	f1_score:0.7947
c1:1.000, c2:0.215 :	f1_score:0.7904
c1:1.000, c2:1.000 :	f1_score:0.7858


In [13]:
# Take best value (approximately) for C1 and C2 from results above:
c1 = 0.05
c2 = 0.2

In [17]:
# Train a model with those parameters:
trainer = Trainer(
    params={
        'c1': c1, 'c2': c2, 'max_iterations': 50
    }, verbose=False)

for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

filename = 'conll2002-esp.crfsuite'
trainer.train(filename)

tagger = Tagger()
tagger.open(filename)

y_pred = [tagger.tag(xseq) for xseq in X_test]
print(bio_classification_report(y_test, y_pred))

             precision    recall  f1-score   support

      B-LOC       0.82      0.77      0.80      1084
      I-LOC       0.76      0.69      0.72       325
     B-MISC       0.68      0.51      0.58       339
     I-MISC       0.68      0.60      0.64       557
      B-ORG       0.81      0.84      0.83      1400
      I-ORG       0.85      0.80      0.82      1104
      B-PER       0.86      0.88      0.87       735
      I-PER       0.90      0.93      0.92       634

avg / total       0.81      0.79      0.80      6178



In [18]:
# Let's check what the classifier learnt
info = tagger.info()

def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

In [19]:
print("Top likely transitions:")
print_transitions(Counter(info.transitions).most_common(15))

print("\nTop unlikely transitions:")
print_transitions(Counter(info.transitions).most_common()[-15:])

Top likely transitions:
B-LOC  -> I-LOC   9.031848
I-MISC -> I-MISC  8.466673
I-LOC  -> I-LOC   8.263667
B-MISC -> I-MISC  7.626786
B-PER  -> I-PER   7.345436
I-PER  -> I-PER   6.139850
B-ORG  -> I-ORG   5.879298
I-ORG  -> I-ORG   4.342204
O      -> B-MISC  2.572779
O      -> B-LOC   2.054747
O      -> B-ORG   1.709448
O      -> O       1.401909
O      -> B-PER   1.244134
B-ORG  -> B-LOC   1.221498
B-LOC  -> B-LOC   0.091866

Top unlikely transitions:
I-MISC -> O       -0.689030
I-PER  -> B-PER   -0.832852
I-ORG  -> B-PER   -0.837975
I-ORG  -> O       -0.962385
I-MISC -> B-LOC   -1.083145
I-ORG  -> B-MISC  -1.130856
B-MISC -> B-PER   -1.132887
B-ORG  -> B-MISC  -1.183964
I-MISC -> B-ORG   -1.331255
B-MISC -> O       -1.473448
I-LOC  -> B-LOC   -1.781599
I-ORG  -> B-ORG   -2.099026
B-PER  -> B-PER   -2.279728
B-ORG  -> B-ORG   -2.324844
I-ORG  -> B-LOC   -2.978606


In [20]:
# Check the state features:
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-6s %s" % (weight, label, attr))    

print("Top positive:")
print_state_features(Counter(info.state_features).most_common(20))

print("\nTop negative:")
print_state_features(Counter(info.state_features).most_common()[-20:])

Top positive:
6.114617 B-ORG  word.lower=efe-cantabria
6.086492 O      word.istitle=False
5.222698 B-ORG  word.lower=psoe-progresistas
4.936052 B-ORG  word.lower=efe
3.765040 O      word.lower=a
3.442109 B-ORG  word.lower=telefónica
3.212038 B-ORG  word[-2:]=-e
3.017436 O      -3:word[-3:]=Día
2.979769 B-ORG  word.lower=ciu
2.979518 B-ORG  word[-2:]=iU
2.979518 B-ORG  word[-3:]=CiU
2.882658 O      word.lower=y
2.878889 B-MISC word.lower=feria
2.816437 O      postag[:2]=Fp
2.792039 B-ORG  word.lower=cámara
2.777382 B-MISC word.lower=diversia
2.696138 B-LOC  word.lower=líbano
2.685301 B-MISC word.lower=justicia
2.676294 B-MISC word.lower=educación
2.672382 B-ORG  word.lower=ejército

Top negative:
-1.583014 O      word[-2:]=nd
-1.644235 O      +3:word.lower=vicente
-1.661696 B-LOC  word[-3:]=rno
-1.668549 O      word.istitle=True
-1.706820 O      -1:word.lower=carretera
-1.716090 O      -3:word.lower=fundación
-1.723849 O      +3:word.lower=principal
-1.725828 B-LOC  word[-3:]=tes
-1.761

In [21]:
def viterbi_decoder(m_xy, n=None, log_version=True):
    """
    Performs MAP inference, determining $y = \argmax_y P(y|x)$, using the
    Viterbi algorithm.

    Parameters
    ----------
    m_xy : ndarray, shape (n_obs, n_labels, n_labels)
        Values of log-potentials ($\log M_i(y_{i-1}, y_i, x)$)
        computed based on feature functions f_xy and/or user-defined potentials
        `psi_xy`. At t=0, m_xy[0, 0, :] contains values of $\log M_1(y_0, y_1)$
        with $y_0$ the fixed initial state.

    n : integer, default=None
        Time position up to which to decode the optimal sequence; if not
        specified (default) the score is computed for the whole sequence.

    Returns
    -------
    y_pred : ndarray, shape (n_obs,)
        Predicted optimal sequence of labels.

    """
    # Initialization
    n_obs, n_labels, _ = m_xy.shape
    delta = np.zeros((n_obs, n_labels))
    psi = np.zeros((n_obs, n_labels))

    delta[0, :] = m_xy[0, 0, :]

    # Recursion
    for m in range(1, n_obs):
        for s in range(n_labels):
            current_exploration = delta[m-1, :] + m_xy[m, :, s]
            psi[m, s] = np.argmax(current_exploration)
            delta[m, s] = current_exploration[psi[m, s]]

    # Termination
    y_pred = np.zeros(n_obs)
    y_pred[-1] = np.argmax(delta[-1, :])

    # Path backtracking
    for m in range(n_obs-2, -1, -1):
        y_pred[m] = psi[m+1, y_pred[m+1]]

    return y_pred.astype(np.int64)

In [23]:
RECREATE = True  # set to True to recreate flexcrf data with new model

CRFSUITE_MODEL_FILE = filename
CRFSUITE_TEST_DATA_FILE = 'conll2002-esp_crfsuite-test-data.dump'
FLEXCRF_TEST_DATA_FILE = 'conll2002-esp_flexcrf-test-data.dump'

# crfsuite model
tagger = Tagger()
tagger.open(CRFSUITE_MODEL_FILE)
model = tagger.info()

data = pickle.load(open(CRFSUITE_TEST_DATA_FILE, 'rb'))
print("test data loaded.")

if RECREATE:
    dataset, thetas = convert_data_to_flexcrf(data, model, n_seq=3)
    pickle.dump({'dataset': dataset, 'thetas': thetas},
                open(FLEXCRF_TEST_DATA_FILE, 'wb'))
else:
    dd = pickle.load(open(FLEXCRF_TEST_DATA_FILE))
    dataset = dd['dataset']
    thetas = dd['thetas']

# -- Start classification ------------------------------------------------

for seq in range(len(dataset)):
    # -- with crfsuite
    s_ = tagger.tag(data['X'][seq])
    y_ = np.array([int(model.labels[s]) for s in s_])
    prob_ = tagger.probability(s_)

    print("-- With CRFSUITE:")
#     print("labels:\n", s_, "\n", y_)
    print("\tprobability:", prob_)

    # -- with flexcrf
    f_xy, y = dataset[seq]
    theta = thetas[seq]

    m_xy, f_m_xy = _compute_all_potentials(f_xy, theta)

    y_pred = viterbi_decoder(m_xy)

    alpha = _forward_score(m_xy=m_xy)
    beta = _backward_score(m_xy=m_xy)
    z_x = _partition_fun_value(alpha=alpha)
    f_x = _feat_fun_values(f_xy=f_xy, y=y_pred, with_f_x_sum=False)

    prob = _posterior_score(f_x=f_x, theta=theta, z_x=z_x, log_version=False)

    print("-- With FLEXCRF:")
#     print("labels:\n", y_pred)
    print("\tprobability:", prob)
    print()
    print("equal predictions: ", np.all(y_pred == y_))
    print("delta:", abs(prob-prob_), '\n\n')
    

tagger.close()


test data loaded.

converting to flexcrf format...
f_xy_desc created.
t_xyy_desc created
Processing sentence 1/3...
Processing sentence 2/3...
Processing sentence 3/3...
-- With CRFSUITE:
	probability: 0.9243423978542037
-- With FLEXCRF:
	probability: 0.924342322718

equal predictions:  True
delta: 7.51357548401e-08 


-- With CRFSUITE:
	probability: 0.9999955324302814
-- With FLEXCRF:
	probability: 0.999995532422

equal predictions:  True
delta: 8.17634848715e-12 


-- With CRFSUITE:
	probability: 0.2266226349829001
-- With FLEXCRF:
	probability: 0.226622571798

equal predictions:  True
delta: 6.31848748878e-08 




In [24]:
train_sents = list(nltk.corpus.conll2000.iob_sents('train.txt'))
test_sents = list(nltk.corpus.conll2000.iob_sents('test.txt'))

X_train = [sent2features(s) for s in train_sents]
y_train = [sent2labels(s) for s in train_sents]
X_test = [sent2features(s) for s in test_sents]
y_test = [sent2labels(s) for s in test_sents]

In [25]:
c1s = np.logspace(-3, 0, 4)
c2s = np.logspace(-3, 0, 4)
c1_c2s = product(c1s, c2s)

with Pool() as p:
    res = p.map(tag, c1_c2s)

In [26]:
for (c1, c2, y_pred) in res:
    lb = LabelBinarizer()
    y_true_combined = lb.fit_transform(list(chain.from_iterable(y_test)))
    y_pred_combined = lb.transform(list(chain.from_iterable(y_pred)))

    tagset = set(lb.classes_) - {'O'}
    tagset = sorted(tagset, key=lambda tag: tag.split('-', 1)[::-1])
    class_indices = {cls: idx for idx, cls in enumerate(lb.classes_)}

    f1score = f1_score(y_true_combined,y_pred_combined,
        labels=[class_indices[cls] for cls in tagset], average='micro')
    print('c1:{0:.3f}, c2:{1:.3f} :\tf1_score:{2:.4f}'.format(c1, c2, f1score))
#     print(bio_classification_report(y_test, y_pred))

c1:0.001, c2:0.001 :	f1_score:0.9415
c1:0.001, c2:0.010 :	f1_score:0.9471
c1:0.001, c2:0.100 :	f1_score:0.9508
c1:0.001, c2:1.000 :	f1_score:0.9524
c1:0.010, c2:0.001 :	f1_score:0.9450
c1:0.010, c2:0.010 :	f1_score:0.9471
c1:0.010, c2:0.100 :	f1_score:0.9511
c1:0.010, c2:1.000 :	f1_score:0.9531
c1:0.100, c2:0.001 :	f1_score:0.9499
c1:0.100, c2:0.010 :	f1_score:0.9499
c1:0.100, c2:0.100 :	f1_score:0.9517
c1:0.100, c2:1.000 :	f1_score:0.9525
c1:1.000, c2:0.001 :	f1_score:0.9536
c1:1.000, c2:0.010 :	f1_score:0.9529
c1:1.000, c2:0.100 :	f1_score:0.9537
c1:1.000, c2:1.000 :	f1_score:0.9538


In [28]:
# Best f1_score parameters from above
c1 = 1.0
c2 = 1.0

# Train a model with those parameters:
trainer = Trainer(
    params={
        'c1': c1, 'c2': c2, 'max_iterations': 50, 
        'feature.possible_transitions': True
    }, verbose=False)

for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

filename = 'conll2002.crfsuite'
trainer.train(filename)

tagger = Tagger()
tagger.open(filename)

y_pred = [tagger.tag(xseq) for xseq in X_test]
print(bio_classification_report(y_test, y_pred))

             precision    recall  f1-score   support

     B-ADJP       0.80      0.74      0.77       438
     I-ADJP       0.80      0.65      0.72       167
     B-ADVP       0.85      0.80      0.82       866
     I-ADVP       0.69      0.57      0.63        89
    B-CONJP       0.71      0.56      0.63         9
    I-CONJP       0.83      0.77      0.80        13
     B-INTJ       1.00      0.50      0.67         2
      B-LST       0.00      0.00      0.00         5
      I-LST       0.00      0.00      0.00         2
       B-NP       0.97      0.96      0.96     12422
       I-NP       0.96      0.97      0.96     14376
       B-PP       0.96      0.98      0.97      4811
       I-PP       0.77      0.62      0.69        48
      B-PRT       0.78      0.77      0.78       106
     B-SBAR       0.90      0.85      0.87       535
     I-SBAR       0.13      0.50      0.21         4
       B-VP       0.95      0.95      0.95      4658
       I-VP       0.94      0.96      0.95   

In [29]:
trainer.logparser.last_iteration

{'active_features': 36349,
 'error_norm': 829.627592,
 'feature_norm': 51.896429,
 'linesearch_step': 1.0,
 'linesearch_trials': 1,
 'loss': 22863.115965,
 'num': 50,
 'scores': {},
 'time': 1.907}